In [1]:
from tqdm.notebook import tqdm
import clip
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

clip_model, clip_preprocess = clip.load("ViT-B/32", device)

cuda


In [2]:
from datasets import *
dataset_obj = Cifar100(0, 50)
train_loader, _ = dataset_obj.get_train_loaders(transform_fn=clip_preprocess,shuffle=False)
test_loader = dataset_obj.get_test_loader(transform_fn=clip_preprocess)
classes = dataset_obj.classes

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [3]:
def get_clip_features(dataset):
    all_features = []
    all_labels = []

    global clip_model

    with torch.no_grad():
        for images, labels in tqdm(dataset):
            features = clip_model.encode_image(images.to(device))
            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features), torch.cat(all_labels)

train_features, train_labels = get_clip_features(train_loader)
test_features, test_labels = get_clip_features(test_loader)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [4]:
def batch(iterable1,iterable2, n=1):
    l = len(iterable1)
    for ndx in range(0, l, n):
        yield (iterable1[ndx:min(ndx + n, l)], iterable2[ndx:min(ndx + n, l)])

out_train = batch(train_features,train_labels, 50)
out_test = batch(test_features,test_labels, 50)

In [5]:
import torch.nn as nn
import torch.optim as optim

class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        outputs = self.linear(x)
        return outputs
    
model = LogisticRegression(512,100)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-5)
num_epochs = 1000
scheduler = optim.lr_scheduler.MultiStepLR(
            optimizer, milestones=[300, 500, 700, 900], gamma=0.1
        )

In [6]:
def num_correct_preds(outputs, labels):
    _, predicted = outputs.max(1)
    return predicted.eq(labels).sum().item()

In [7]:
import random

model.to(device)
model.train()
model = model.to(torch.float16)

for epoch in range(num_epochs):
    
    running_loss = 0.0
    correct = 0.0
    total = 0
    c = list(zip(train_features, train_labels))
    random.shuffle(c)
    train_features, train_labels = zip(*c)
    out_train = batch(train_features,train_labels, 50)
    for inputs, labels in tqdm(out_train):
        
        inputs = inputs / inputs.norm(dim=-1, keepdim=True)
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()

        optimizer.step()
        running_loss+=loss
        total += len(labels)
        correct += num_correct_preds(outputs, labels)
        
        
    epoch_loss = running_loss/len(train_loader)
    epoch_accuracy = correct*100/total
    print(
        f"Training: Epoch {epoch} || Loss: {epoch_loss:7.3f} || Accuracy: {epoch_accuracy:6.2f}%"
    )
    scheduler.step()

0it [00:00, ?it/s]

AttributeError: 'tuple' object has no attribute 'norm'

In [ ]:
global clip_model, clip_preprocess
from sklearn.linear_model import LogisticRegression

len_classes = len(classes)

train_features = train_features / train_features.norm(dim=-1, keepdim=True)
test_features = test_features / test_features.norm(dim=-1, keepdim=True)

classifier = LogisticRegression(C=0.1, max_iter=1000, n_jobs=4,verbose=1)
classifier.fit(train_features.cpu().numpy(), train_labels.cpu().numpy())
predictions = classifier.predict(train_features.cpu().numpy())
accuracy = np.mean((train_labels.cpu().numpy() == predictions).astype(np.float)) * 100.0

print(accuracy)